# Feed-Forward Neural Network

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [2]:
from itertools import product
from pathlib import Path
from tqdm.auto import trange

import numpy as np
import xarray as xr
import xarray.ufuncs as xf
import matplotlib.pyplot as plt
from scipy.interpolate import griddata

from system_identification.ffnn import FeedForwardNeuralNetwork, TrainingParameters
from system_identification.load_assignment_data import load_net_example_ff

In [3]:
nn = FeedForwardNeuralNetwork.new(
    n_inputs=2,
    n_outputs=1,
    n_hidden=124,
    range=[[-1, 1], [-1, 1]],
    log_dir="./ffnn_1",
    training_parameters=TrainingParameters(
        epochs=1000,
        goal=0,
        min_grad=1e-10,
        mu=0.001,
    ),
)

In [4]:
data = xr.open_dataset("data_smoothed.nc")
data

<xarray.Dataset>
Dimensions:                 (dim_0: 4, dim_1: 4, t: 10001, x_idx: 4)
Coordinates:
  * t                       (t) float64 0.0 0.01 0.02 0.03 ... 99.98 99.99 100.0
Dimensions without coordinates: dim_0, dim_1, x_idx
Data variables: (12/35)
    p_k1k1                  (t, dim_0, dim_1) float64 ...
    p_kk1                   (t, dim_0, dim_1) float64 ...
    phi                     (t, dim_0, dim_1) float64 ...
    gamma                   (t, dim_0, dim_1) float64 ...
    ps                      (t, dim_0, dim_1) float64 ...
    x_k1k1                  (t, x_idx) float64 ...
    ...                      ...
    w_smoothed_std          (t) float64 ...
    C_alpha_u_smoothed      (t) float64 ...
    C_alpha_u_smoothed_std  (t) float64 ...
    vm_filtered             (t) float64 ...
    vm_smoothed             (t) float64 ...
    alpha_estimate          (t) float64 ...

In [5]:
inputs = np.hstack((
    np.expand_dims(data.alpha_estimate.values, 1),
    np.expand_dims(data.beta_m.values, 1)
))

reference_outputs = np.expand_dims(data.c_m.values, 1)

In [6]:
nn.save()

In [7]:
nn.back_propagation(inputs, reference_outputs, epochs=1000)

  0%|          | 0/1000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [66]:
# print(nn.input_weights)
# print(nn.output_weights)
# print(nn.bias_weights)

coords = np.linspace(-5, 5, 100)
nn_eval = np.empty((100,))
fn_eval = np.empty((100,))
plt.figure()

for y in np.linspace(-1, 1, 4):
    for xi, x in enumerate(coords):
        nn_eval[xi] = nn.evaluate([x, y])

    plt.plot(coords, fn_eval, "b")
    plt.plot(coords, nn_eval, "r")

# plt.plot(inputs, reference_outputs, ".", markersize=.1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [76]:
alpha_grid = np.linspace(-0.2, 0.8, 1000)
beta_grid = np.linspace(-0.3, 0.3, 1000)
clim = min(reference_outputs), max(reference_outputs)

nn_eval = np.empty(reference_outputs.shape)
for ii, input in enumerate(inputs):
    nn_eval[ii] = nn.evaluate(input)

fig = plt.figure()
ax = plt.gca()
zi = griddata((inputs[:, 0], inputs[:, 1]), nn_eval, (alpha_grid[None, :], beta_grid[:, None]), method='linear')
pos = plt.imshow(zi, cmap='viridis')
fig.colorbar(pos)
plt.clim(*clim)

fig = plt.figure()
ax = plt.gca()
zi = griddata((inputs[:, 0], inputs[:, 1]), reference_outputs, (alpha_grid[None, :], beta_grid[:, None]), method='linear')
pos = plt.imshow(zi, cmap='viridis')
fig.colorbar(pos)
plt.clim(*clim)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(inputs[:, 0], inputs[:, 1], reference_outputs)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(inputs[:, 0], inputs[:, 1], nn_eval)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

ShapeError: Matrix size mismatch: (3, 2) * (1, 3).